# NN with OneHotEncode Target Values
Four Binary outputs

Performs better than NN with LabelEncode Target Values

In [2]:
#Import Dependencies

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import os
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import session
from sqlalchemy import create_engine, func
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf

In [4]:
#SOURCE: https://www.tutorialspoint.com/python_data_access/python_postgresql_database_connection.htm
import psycopg2

#establishing the connection
conn = psycopg2.connect(
   database="Body_Performance", user='postgres', password='Challah6132', host='localhost', port= '5432'
)
#Creating a cursor object using the cursor() method
cursor = conn.cursor()

#Executing an MYSQL function using the execute() method
cursor.execute("select version()")

# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print("Connection established to: ",data)



Connection established to:  ('PostgreSQL 12.13, compiled by Visual C++ build 1914, 64-bit',)


In [5]:
body_df = pd.read_sql('select * from body_performance', con=conn)
body_df.head()

,age,gender,height_cm,weight_kg,body_fat_pct,diastolic,systolic,gripforce,sit_and_bend_forward_cm,sit_ups_counts,broad_jump_cm,ranking
0,27,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C
1,25,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A
2,31,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C
3,32,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B
4,28,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B


In [6]:
#Closing the connection
conn.close()
print("Closed connection established to: ",data)


Closed connection established to:  ('PostgreSQL 12.13, compiled by Visual C++ build 1914, 64-bit',)


In [7]:
#LabelEncoder on gender column
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()

body_df['gender_num'] = labelencoder.fit_transform(body_df['gender'])

body_df.head(10)

,age,gender,height_cm,weight_kg,body_fat_pct,diastolic,systolic,gripforce,sit_and_bend_forward_cm,sit_ups_counts,broad_jump_cm,ranking,gender_num
0,27,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C,1
1,25,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A,1
2,31,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C,1
3,32,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B,1
4,28,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B,1
5,36,F,165.4,55.40,22.0,64.0,119.0,23.8,21.0,27.0,153.0,B,0
6,42,F,164.5,63.70,32.2,72.0,135.0,22.7,0.8,18.0,146.0,D,0
7,33,M,174.9,77.20,36.9,84.0,137.0,45.9,12.3,42.0,234.0,B,1
8,54,M,166.8,67.50,27.6,85.0,165.0,40.4,18.6,34.0,148.0,C,1
9,28,M,185.0,84.60,14.4,81.0,156.0,57.9,12.1,55.0,213.0,B,1


In [9]:
#Using oneHotEncoder for class into 4 separate
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)

encode2 = pd.DataFrame(enc.fit_transform(body_df.ranking.values.reshape(-1,1)))

encode2.columns = enc.get_feature_names_out(["ranking"])
encode2.head()

,ranking_A,ranking_B,ranking_C,ranking_D
0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0


In [10]:
#Drop columns and merge encoded df

body_df = body_df.drop(["gender", "ranking"], axis=1)

body_df.head()


,age,height_cm,weight_kg,body_fat_pct,diastolic,systolic,gripforce,sit_and_bend_forward_cm,sit_ups_counts,broad_jump_cm,gender_num
0,27,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,1
1,25,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,1
2,31,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,1
3,32,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,1
4,28,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,1


In [11]:
#Set X, y 
X = body_df.values
y = encode2.values
print(X.shape, y.shape)


(13393, 11) (13393, 4)


In [12]:
y[:10]

array([[0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]])

In [13]:
#Split train/test data and scale using Standard Scaler at this time (mean of 0).  #Should we use MinMax(0-1)
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

scaler = StandardScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(X_train_scaled.shape, X_test_scaled.shape)

(10044, 11) (3349, 11)


In [14]:
# Define the model - deep neural net...length of outputs/inputs
num_input_features = len(X_train_scaled[0])
num_output_features = len(y_train[0])
print(num_input_features)
print(num_output_features)

11
4


### Test One
- 3 hidden layers
- equal nodes (10 each)
- 200 epochs
- relu, sig, sig, soft
- binary_crossentropy

In [15]:
# Define the model - deep neural net
# Try with 2 hidden layers first but will try reduced. Look at different activation functions?
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 10
hidden_nodes_layer3 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer1, input_dim=num_input_features,
    activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer  #Will need to round or should this be sigmoid?
nn.add(tf.keras.layers.Dense(units=num_output_features, activation="softmax"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                120       
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 10)                110       
                                                                 
 dense_3 (Dense)             (None, 4)                 44        
                                                                 
Total params: 384
Trainable params: 384
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model binary vs categorical. Categorical for multi-output  --https://machinelearningmastery.com/deep-learning-models-for-multi-output-regression/
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
#nn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
314/314 [==============================] - 1s 799us/step - loss: 0.6945 - accuracy: 0.2512
Epoch 2/200
314/314 [==============================] - 0s 779us/step - loss: 0.5372 - accuracy: 0.4469
Epoch 3/200
314/314 [==============================] - 0s 808us/step - loss: 0.4851 - accuracy: 0.5112
Epoch 4/200
314/314 [==============================] - 0s 790us/step - loss: 0.4540 - accuracy: 0.5312
Epoch 5/200
314/314 [==============================] - 0s 785us/step - loss: 0.4358 - accuracy: 0.5638
Epoch 6/200
314/314 [==============================] - 0s 795us/step - loss: 0.4226 - accuracy: 0.5866
Epoch 7/200
314/314 [==============================] - 0s 779us/step - loss: 0.4108 - accuracy: 0.6021
Epoch 8/200
314/314 [==============================] - 0s 779us/step - loss: 0.4001 - accuracy: 0.6122
Epoch 9/200
314/314 [==============================] - 0s 798us/step - loss: 0.3902 - accuracy: 0.6230
Epoch 10/200
314/314 [==============================] - 0s 785us/step - l

314/314 [==============================] - 0s 773us/step - loss: 0.2707 - accuracy: 0.7532
Epoch 158/200
314/314 [==============================] - 0s 777us/step - loss: 0.2705 - accuracy: 0.7530
Epoch 159/200
314/314 [==============================] - 0s 787us/step - loss: 0.2703 - accuracy: 0.7519
Epoch 160/200
314/314 [==============================] - 0s 779us/step - loss: 0.2704 - accuracy: 0.7539
Epoch 161/200
314/314 [==============================] - 0s 776us/step - loss: 0.2703 - accuracy: 0.7545
Epoch 162/200
314/314 [==============================] - 0s 774us/step - loss: 0.2705 - accuracy: 0.7532
Epoch 163/200
314/314 [==============================] - 0s 785us/step - loss: 0.2702 - accuracy: 0.7524
Epoch 164/200
314/314 [==============================] - 0s 776us/step - loss: 0.2701 - accuracy: 0.7520
Epoch 165/200
314/314 [==============================] - 0s 798us/step - loss: 0.2702 - accuracy: 0.7556
Epoch 166/200
314/314 [==============================] - 0s 781us/ste

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

105/105 - 0s - loss: 0.2816 - accuracy: 0.7381 - 167ms/epoch - 2ms/step
Loss: 0.28157320618629456, Accuracy: 0.7381308078765869


### Test Two
- 3 hidden layers
- nodes (14, 12, 8)
- 200 epochs
- relu, sig, sig, soft
- binary_crossentropy# second model test

In [21]:
# New Layer node count

hidden2_nodes_layer1 =  14
hidden2_nodes_layer2 = 12
hidden2_nodes_layer3 = 8
#2hidden_nodes_layer4 = 6

In [22]:

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(
    units=hidden2_nodes_layer1, input_dim=num_input_features,
    activation="relu"))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden2_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden2_nodes_layer3, activation="sigmoid"))

# Fourth hidden layer
#nn2.add(tf.keras.layers.Dense(units=hidden2_nodes_layer4, activation="sigmoid"))

# Output layer  #Will need to round or should this be sigmoid?
nn2.add(tf.keras.layers.Dense(units=num_output_features, activation="softmax"))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 14)                168       
                                                                 
 dense_5 (Dense)             (None, 12)                180       
                                                                 
 dense_6 (Dense)             (None, 8)                 104       
                                                                 
 dense_7 (Dense)             (None, 4)                 36        
                                                                 
Total params: 488
Trainable params: 488
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [24]:
# Train the model
fit_model = nn2.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
314/314 [==============================] - 1s 780us/step - loss: 0.6176 - accuracy: 0.2576
Epoch 2/200
314/314 [==============================] - 0s 780us/step - loss: 0.5114 - accuracy: 0.4174
Epoch 3/200
314/314 [==============================] - 0s 804us/step - loss: 0.4620 - accuracy: 0.5335
Epoch 4/200
314/314 [==============================] - 0s 837us/step - loss: 0.4379 - accuracy: 0.5519
Epoch 5/200
314/314 [==============================] - 0s 954us/step - loss: 0.4191 - accuracy: 0.5932
Epoch 6/200
314/314 [==============================] - 0s 866us/step - loss: 0.4025 - accuracy: 0.6157
Epoch 7/200
314/314 [==============================] - 0s 872us/step - loss: 0.3878 - accuracy: 0.6345
Epoch 8/200
314/314 [==============================] - 0s 850us/step - loss: 0.3747 - accuracy: 0.6544
Epoch 9/200
314/314 [==============================] - 0s 795us/step - loss: 0.3634 - accuracy: 0.6682
Epoch 10/200
314/314 [==============================] - 0s 818us/step - l

314/314 [==============================] - 0s 781us/step - loss: 0.2659 - accuracy: 0.7610
Epoch 158/200
314/314 [==============================] - 0s 788us/step - loss: 0.2656 - accuracy: 0.7619
Epoch 159/200
314/314 [==============================] - 0s 778us/step - loss: 0.2655 - accuracy: 0.7614
Epoch 160/200
314/314 [==============================] - 0s 782us/step - loss: 0.2649 - accuracy: 0.7623
Epoch 161/200
314/314 [==============================] - 0s 784us/step - loss: 0.2659 - accuracy: 0.7605
Epoch 162/200
314/314 [==============================] - 0s 795us/step - loss: 0.2652 - accuracy: 0.7625
Epoch 163/200
314/314 [==============================] - 0s 777us/step - loss: 0.2656 - accuracy: 0.7614
Epoch 164/200
314/314 [==============================] - 0s 773us/step - loss: 0.2653 - accuracy: 0.7625
Epoch 165/200
314/314 [==============================] - 0s 792us/step - loss: 0.2651 - accuracy: 0.7624
Epoch 166/200
314/314 [==============================] - 0s 812us/ste

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

#One Hidden Layer lowered accuracy and inreased loss. 

105/105 - 0s - loss: 0.2831 - accuracy: 0.7372 - 138ms/epoch - 1ms/step
Loss: 0.2831159830093384, Accuracy: 0.7372350096702576


### Test Three

### Best Neural Network Model
- 3 hidden layers
- nodes (14, 10, 8)
- 200 epochs
- relu, sig, sig, soft
- binary_crossentropy# second model test

In [43]:
# New Layer node count

hidden3_nodes_layer1 =  14
hidden3_nodes_layer2 = 10
hidden3_nodes_layer3 = 8
#2hidden_nodes_layer4 = 6

In [29]:
nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(
    units=hidden3_nodes_layer1, input_dim=num_input_features,
    activation="relu"))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden3_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden3_nodes_layer3, activation="sigmoid"))

# Fourth hidden layer
#nn2.add(tf.keras.layers.Dense(units=hidden3_nodes_layer4, activation="sigmoid"))

# Output layer  #Will need to round or should this be sigmoid?
nn3.add(tf.keras.layers.Dense(units=num_output_features, activation="softmax"))

# Check the structure of the model
nn3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 14)                168       
                                                                 
 dense_10 (Dense)            (None, 10)                150       
                                                                 
 dense_11 (Dense)            (None, 8)                 88        
                                                                 
 dense_12 (Dense)            (None, 4)                 36        
                                                                 
Total params: 442
Trainable params: 442
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [57]:
# Train the model
import time
start_time = time.time()
fit_model = nn3.fit(X_train_scaled,y_train,epochs=200)
print(f'This model took, {time.time() - start_time} to run')

Epoch 1/200
314/314 [==============================] - 0s 801us/step - loss: 0.2612 - accuracy: 0.7641
Epoch 2/200
314/314 [==============================] - 0s 823us/step - loss: 0.2618 - accuracy: 0.7610
Epoch 3/200
314/314 [==============================] - 0s 839us/step - loss: 0.2612 - accuracy: 0.7623
Epoch 4/200
314/314 [==============================] - 0s 824us/step - loss: 0.2613 - accuracy: 0.7626
Epoch 5/200
314/314 [==============================] - 0s 788us/step - loss: 0.2612 - accuracy: 0.7629
Epoch 6/200
314/314 [==============================] - 0s 806us/step - loss: 0.2613 - accuracy: 0.7620
Epoch 7/200
314/314 [==============================] - 0s 785us/step - loss: 0.2612 - accuracy: 0.7628
Epoch 8/200
314/314 [==============================] - 0s 826us/step - loss: 0.2613 - accuracy: 0.7619
Epoch 9/200
314/314 [==============================] - 0s 838us/step - loss: 0.2616 - accuracy: 0.7626
Epoch 10/200
314/314 [==============================] - 0s 805us/step - l

314/314 [==============================] - 0s 786us/step - loss: 0.2563 - accuracy: 0.7682
Epoch 158/200
314/314 [==============================] - 0s 789us/step - loss: 0.2566 - accuracy: 0.7690
Epoch 159/200
314/314 [==============================] - 0s 805us/step - loss: 0.2562 - accuracy: 0.7696
Epoch 160/200
314/314 [==============================] - 0s 833us/step - loss: 0.2563 - accuracy: 0.7706
Epoch 161/200
314/314 [==============================] - 0s 809us/step - loss: 0.2561 - accuracy: 0.7707
Epoch 162/200
314/314 [==============================] - 0s 799us/step - loss: 0.2563 - accuracy: 0.7675
Epoch 163/200
314/314 [==============================] - 0s 804us/step - loss: 0.2566 - accuracy: 0.7706
Epoch 164/200
314/314 [==============================] - 0s 804us/step - loss: 0.2558 - accuracy: 0.7713
Epoch 165/200
314/314 [==============================] - 0s 792us/step - loss: 0.2563 - accuracy: 0.7687
Epoch 166/200
314/314 [==============================] - 0s 808us/ste

In [58]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

#One Hidden Layer lowered accuracy and inreased loss. 

105/105 - 0s - loss: 0.2818 - accuracy: 0.7375 - 74ms/epoch - 706us/step
Loss: 0.2817964255809784, Accuracy: 0.7375335693359375


### Test Four
- 3 hidden layers
- nodes (12, 10, 8)
- 200 epochs
- relu, sig, sig, soft
- binary_crossentropy# second model test

In [35]:
# New Layer node count

hidden4_nodes_layer1 =  12
hidden4_nodes_layer2 = 10
hidden4_nodes_layer3 = 8
#2hidden_nodes_layer4 = 6

In [36]:
nn4 = tf.keras.models.Sequential()

# First hidden layer
nn4.add(tf.keras.layers.Dense(
    units=hidden4_nodes_layer1, input_dim=num_input_features,
    activation="relu"))

# Second hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden4_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden4_nodes_layer3, activation="sigmoid"))

# Fourth hidden layer
#nn4.add(tf.keras.layers.Dense(units=hidden4_nodes_layer4, activation="sigmoid"))

# Output layer  #Will need to round or should this be sigmoid?
nn4.add(tf.keras.layers.Dense(units=num_output_features, activation="softmax"))

# Check the structure of the model
nn4.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 12)                144       
                                                                 
 dense_14 (Dense)            (None, 10)                130       
                                                                 
 dense_15 (Dense)            (None, 8)                 88        
                                                                 
 dense_16 (Dense)            (None, 4)                 36        
                                                                 
Total params: 398
Trainable params: 398
Non-trainable params: 0
_________________________________________________________________


In [38]:
# Compile the model
nn4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [39]:
# Train the model
fit_model = nn4.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
314/314 [==============================] - 1s 842us/step - loss: 0.6096 - accuracy: 0.2507
Epoch 2/200
314/314 [==============================] - 0s 787us/step - loss: 0.5344 - accuracy: 0.4151
Epoch 3/200
314/314 [==============================] - 0s 797us/step - loss: 0.4832 - accuracy: 0.5120
Epoch 4/200
314/314 [==============================] - 0s 778us/step - loss: 0.4508 - accuracy: 0.5375
Epoch 5/200
314/314 [==============================] - 0s 784us/step - loss: 0.4285 - accuracy: 0.5840
Epoch 6/200
314/314 [==============================] - 0s 781us/step - loss: 0.4097 - accuracy: 0.6110
Epoch 7/200
314/314 [==============================] - 0s 779us/step - loss: 0.3942 - accuracy: 0.6303
Epoch 8/200
314/314 [==============================] - 0s 799us/step - loss: 0.3818 - accuracy: 0.6458
Epoch 9/200
314/314 [==============================] - 0s 822us/step - loss: 0.3718 - accuracy: 0.6559
Epoch 10/200
314/314 [==============================] - 0s 853us/step - l

314/314 [==============================] - 0s 823us/step - loss: 0.2677 - accuracy: 0.7602
Epoch 158/200
314/314 [==============================] - 0s 775us/step - loss: 0.2675 - accuracy: 0.7594
Epoch 159/200
314/314 [==============================] - 0s 799us/step - loss: 0.2676 - accuracy: 0.7601
Epoch 160/200
314/314 [==============================] - 0s 831us/step - loss: 0.2677 - accuracy: 0.7567
Epoch 161/200
314/314 [==============================] - 0s 779us/step - loss: 0.2673 - accuracy: 0.7578
Epoch 162/200
314/314 [==============================] - 0s 857us/step - loss: 0.2675 - accuracy: 0.7587
Epoch 163/200
314/314 [==============================] - 0s 893us/step - loss: 0.2673 - accuracy: 0.7584
Epoch 164/200
314/314 [==============================] - 0s 847us/step - loss: 0.2673 - accuracy: 0.7591
Epoch 165/200
314/314 [==============================] - 0s 788us/step - loss: 0.2671 - accuracy: 0.7564
Epoch 166/200
314/314 [==============================] - 0s 876us/ste

In [40]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

#One Hidden Layer lowered accuracy and inreased loss. 

105/105 - 0s - loss: 0.2843 - accuracy: 0.7366 - 144ms/epoch - 1ms/step
Loss: 0.28433775901794434, Accuracy: 0.7366378307342529


### Test Five
- 3 hidden layers
- nodes (14, 10, 8)
- 200 epochs
- relu, sig, sig, soft
- categorical_crossentropy

In [53]:
nn5 = tf.keras.models.Sequential()

# First hidden layer
nn5.add(tf.keras.layers.Dense(
    units=hidden3_nodes_layer1, input_dim=num_input_features,
    activation="relu"))

# Second hidden layer
nn5.add(tf.keras.layers.Dense(units=hidden3_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn5.add(tf.keras.layers.Dense(units=hidden3_nodes_layer3, activation="sigmoid"))

# Output layer  #Will need to round or should this be sigmoid?
nn5.add(tf.keras.layers.Dense(units=num_output_features, activation="softmax"))

# Check the structure of the model
nn5.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_37 (Dense)            (None, 14)                168       
                                                                 
 dense_38 (Dense)            (None, 10)                150       
                                                                 
 dense_39 (Dense)            (None, 8)                 88        
                                                                 
 dense_40 (Dense)            (None, 4)                 36        
                                                                 
Total params: 442
Trainable params: 442
Non-trainable params: 0
_________________________________________________________________


In [54]:
# Compile the model
nn5.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [55]:
# Train the model
fit_model = nn5.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
314/314 [==============================] - 1s 756us/step - loss: 1.3519 - accuracy: 0.3470
Epoch 2/200
314/314 [==============================] - 0s 740us/step - loss: 1.1463 - accuracy: 0.5093
Epoch 3/200
314/314 [==============================] - 0s 743us/step - loss: 0.9928 - accuracy: 0.5745
Epoch 4/200
314/314 [==============================] - 0s 750us/step - loss: 0.9230 - accuracy: 0.6022
Epoch 5/200
314/314 [==============================] - 0s 768us/step - loss: 0.8851 - accuracy: 0.6162
Epoch 6/200
314/314 [==============================] - 0s 751us/step - loss: 0.8573 - accuracy: 0.6288
Epoch 7/200
314/314 [==============================] - 0s 748us/step - loss: 0.8330 - accuracy: 0.6405
Epoch 8/200
314/314 [==============================] - 0s 739us/step - loss: 0.8096 - accuracy: 0.6568
Epoch 9/200
314/314 [==============================] - 0s 746us/step - loss: 0.7893 - accuracy: 0.6714
Epoch 10/200
314/314 [==============================] - 0s 742us/step - l

314/314 [==============================] - 0s 750us/step - loss: 0.5907 - accuracy: 0.7587
Epoch 158/200
314/314 [==============================] - 0s 725us/step - loss: 0.5909 - accuracy: 0.7587
Epoch 159/200
314/314 [==============================] - 0s 723us/step - loss: 0.5917 - accuracy: 0.7570
Epoch 160/200
314/314 [==============================] - 0s 717us/step - loss: 0.5906 - accuracy: 0.7564
Epoch 161/200
314/314 [==============================] - 0s 742us/step - loss: 0.5905 - accuracy: 0.7560
Epoch 162/200
314/314 [==============================] - 0s 735us/step - loss: 0.5908 - accuracy: 0.7576
Epoch 163/200
314/314 [==============================] - 0s 742us/step - loss: 0.5902 - accuracy: 0.7575
Epoch 164/200
314/314 [==============================] - 0s 731us/step - loss: 0.5907 - accuracy: 0.7578
Epoch 165/200
314/314 [==============================] - 0s 742us/step - loss: 0.5900 - accuracy: 0.7585
Epoch 166/200
314/314 [==============================] - 0s 751us/ste

In [56]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn5.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

#One Hidden Layer lowered accuracy and inreased loss. 

105/105 - 0s - loss: 0.6313 - accuracy: 0.7453 - 143ms/epoch - 1ms/step
Loss: 0.6312924027442932, Accuracy: 0.7452970743179321
